---
description: We define all the usefull functions to encode the TR sequence as an imput
  for the predictive model
output-file: encoding.html
title: encoding

---

In [ ]:
#| default_exp encoding

In [ ]:
# | export
import ViennaRNA as RNA
import numpy as np
from tqdm import tqdm

In [ ]:
# |export
def encode_tr_list(list_TRs: list # A list of DNA sequences (strings) to encode.
                         ):
    """
    Encodes a list of TR sequences.
    """
    Avd='aagggcaggctgggaaATAA'.upper()
    Sp='tctgtgcccatcaccttcttgcatggctctgccaacgctacggcttggcgggctggcctttcctcaataggtggtcagccggttctgtcctgcttcggcgaacacgttacacggttcggcaaaacgtcgattactgaaaatggaaaggcggggccgacttc'.upper()

    # Initialize an empty list to store the encoded features
    new_features_encoded = []

    # Iterate through each DNA sequence in the list
    for TR in list_TRs:
        features = []

        # Base folding energy of TR alone
        fc_base = RNA.fold_compound(TR)
        _, e_tr = fc_base.pf()
    
        fc_sp = RNA.fold_compound(TR + Sp[:30])
        _, e_tr_sp = fc_sp.pf()
        features.append(e_tr_sp - e_tr)
        new_features_encoded.append(features)

    # Convert the list of encoded features into a NumPy array
    return np.array(new_features_encoded)

In [ ]:
TR_list=['AACTTGCTAGTAAACACAGTGCAGCTAAAGTGCATTCTAATGGGGTACTAAATGTGTTGTTCCATCTGGG','CCAGAGGTTAGGATTATGAGTGTAGTGCGTATCACGTGGGTTGCAGATTAAACAGGTCGTGCCTCATTTA']
encode_tr_list(TR_list)

array([[ -8.17911339],
       [-11.47970581]])

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()